In [1]:
import os
from huggingface_hub import login
token = os.getenv("GITHUB_TOKEN")
login(token=token)

In [2]:
import os, yaml, sys
import torch
from transformers import pipeline
from transformers import logging
logging.set_verbosity_error()
from huggingface_hub import login
token = os.getenv("GITHUB_TOKEN")
login(token=token)
pipe = pipeline(
    task="image-feature-extraction",
    model="facebook/dinov3-vits16-pretrain-lvd1689m",
    dtype=torch.bfloat16,
)

f = pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

W1213 18:46:29.955000 69009 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
import torch
from transformers import AutoImageProcessor, AutoModel
from transformers.image_utils import load_image
import accelerate
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = load_image(url)

processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m")
model = AutoModel.from_pretrained(
    "facebook/dinov3-vits16-pretrain-lvd1689m",
    dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa"
)

inputs = processor(images=image, return_tensors="pt").to(model.device)
with torch.inference_mode():
    outputs = model(**inputs)

pooled_output = outputs.pooler_output
print("Pooled output shape:", pooled_output.shape)

Pooled output shape: torch.Size([1, 384])


In [ ]:
# the usual feature extractor doesn't work
from torchvision.models.feature_extraction import (
    create_feature_extractor,
    get_graph_node_names,
)
get_graph_node_names(model)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nw/yf48zdjj1m5012281wx_pzhc0000gn/T/ipykernel_69009/3213110985.py", line 5, in <module>
    get_graph_node_names(model)
  File "/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/torchvision/models/feature_extraction.py", line 257, in get_graph_node_names
    train_tracer.trace(model.train(), concrete_args=concrete_args)
  File "/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/torch/fx/_symbolic_trace.py", line 850, in trace
    (self.create_arg(fn(*args)),),
                     ^^^^^^^^^
  File "/Users/tizianocausin/Desktop/virtual_envs/temporal_context/lib/python3.12/site-packages/transformers/utils/generic.py", line 826, in wrapper
    "`use_cache=

In [5]:
# Print model modules and parameter shapes (uses existing `model` and `torch` from the notebook)
def print_model_layer_shapes(model):
    total_params = 0
    trainable_params = 0

    for module_name, module in model.named_modules():
        # skip the top-level empty name unless it has no parent info to show
        params = list(module.named_parameters(recurse=False))
        if not params:
            continue

        print(f"Module: '{module_name}' ({module.__class__.__name__})")
        for p_name, p in params:
            shape = tuple(p.shape)
            numel = p.numel()
            total_params += numel
            if p.requires_grad:
                trainable_params += numel
            print(f"  param: {p_name:20s} shape={str(shape):20s} numel={numel:10d} dtype={p.dtype}")
        print()

    print(f"Total params: {total_params:,}")
    print(f"Trainable params: {trainable_params:,}")

# Run the printer
print_model_layer_shapes(model)

Module: 'embeddings' (DINOv3ViTEmbeddings)
  param: cls_token            shape=(1, 1, 384)          numel=       384 dtype=torch.float16
  param: mask_token           shape=(1, 1, 384)          numel=       384 dtype=torch.float16
  param: register_tokens      shape=(1, 4, 384)          numel=      1536 dtype=torch.float16

Module: 'embeddings.patch_embeddings' (Conv2d)
  param: weight               shape=(384, 3, 16, 16)     numel=    294912 dtype=torch.float16
  param: bias                 shape=(384,)               numel=       384 dtype=torch.float16

Module: 'layer.0.norm1' (LayerNorm)
  param: weight               shape=(384,)               numel=       384 dtype=torch.float16
  param: bias                 shape=(384,)               numel=       384 dtype=torch.float16

Module: 'layer.0.attention.k_proj' (Linear)
  param: weight               shape=(384, 384)           numel=    147456 dtype=torch.float16

Module: 'layer.0.attention.v_proj' (Linear)
  param: weight               

In [9]:
# Dictionary to store features
features = {}

def hook_fn(module, input, output):
    features["layer0_scale1"] = output.detach()

# Register hook
target_layer = model.layer[0].layer_scale1
hook_handle = target_layer.register_forward_hook(hook_fn)

In [13]:
print(inputs['pixel_values'].shape)

torch.Size([1, 3, 224, 224])


In [ ]:
inputs = processor(images=image, return_tensors="pt").to(model.device)

with torch.inference_mode():    
    outputs = model(**inputs)

# Extract the saved feature
feat = features["layer0_scale1"]
print("Hooked feature shape:", feat.shape)

Hooked feature shape: torch.Size([1, 201, 384])
